In [1]:
# analyze_anomalies.py
import pandas as pd
from utils import load_messages

def analyze_member_data():
    # 1️⃣ Load messages using existing utils
    messages = load_messages()
    df = pd.DataFrame(messages)
    print(f"✅ Loaded {len(df)} total messages.")

    # 2️⃣ Basic columns check
    expected_cols = ["id", "message", "user_name", "user_id", "timestamp"]
    print("\n📋 Columns present:", list(df.columns))
    missing_cols = [c for c in expected_cols if c not in df.columns]
    if missing_cols:
        print(f"⚠️ Missing expected columns: {missing_cols}")

    # 3️⃣ Check missing values
    print("\n🔍 Missing value counts:")
    print(df.isna().sum())

    # 4️⃣ Empty or whitespace messages
    empty_msgs = df[df["message"].str.strip() == ""]
    print(f"\n⚠️ Empty or whitespace-only messages: {len(empty_msgs)}")

    # 5️⃣ Duplicate checks
    dup_ids = df[df["id"].duplicated(keep=False)]
    dup_texts = df[df["message"].duplicated(keep=False)]
    print(f"\n🌀 Duplicate IDs: {len(dup_ids)}")
    print(f"🌀 Duplicate message texts: {len(dup_texts)}")

    # 6️⃣ User inconsistencies
    user_inconsistencies = (
        df.groupby("user_id")["user_name"].nunique().reset_index(name="unique_names")
    )
    multi_name_users = user_inconsistencies[user_inconsistencies["unique_names"] > 1]
    print(f"\n⚖️ Users with multiple names: {len(multi_name_users)}")

    # 7️⃣ Timestamp issues
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    invalid_timestamps = df["timestamp"].isna().sum()
    # 7️⃣ Timestamp issues (fixed timezone mismatch)
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
    invalid_timestamps = df["timestamp"].isna().sum()
    future_timestamps = (df["timestamp"] > pd.Timestamp.now(tz="UTC")).sum()
    print(f"\n⏳ Invalid timestamps: {invalid_timestamps}")
    print(f"🚀 Future timestamps: {future_timestamps}")

    # 8️⃣ Message length outliers
    df["msg_len"] = df["message"].str.len()
    long_msgs = df[df["msg_len"] > 1000]
    short_msgs = df[df["msg_len"] < 3]
    print(f"\n📏 Extremely long messages (>1000 chars): {len(long_msgs)}")
    print(f"📏 Extremely short messages (<3 chars): {len(short_msgs)}")

    # 9️⃣ Summary for README
    summary = {
        "total_messages": len(df),
        "missing_messages": len(empty_msgs),
        "duplicate_ids": len(dup_ids),
        "duplicate_texts": len(dup_texts),
        "user_inconsistencies": len(multi_name_users),
        "invalid_timestamps": int(invalid_timestamps),
        "future_timestamps": int(future_timestamps),
        "outlier_short_msgs": len(short_msgs),
        "outlier_long_msgs": len(long_msgs),
    }

    print("\n✅ Summary:")
    for k, v in summary.items():
        print(f"  - {k.replace('_', ' ').title()}: {v}")

    return summary


if __name__ == "__main__":
    analyze_member_data()

💾 Loaded 3349 messages from cache.
✅ Loaded 3349 total messages.

📋 Columns present: ['id', 'user_id', 'user_name', 'timestamp', 'message']

🔍 Missing value counts:
id           0
user_id      0
user_name    0
timestamp    0
message      0
dtype: int64

⚠️ Empty or whitespace-only messages: 0

🌀 Duplicate IDs: 0
🌀 Duplicate message texts: 0

⚖️ Users with multiple names: 0

⏳ Invalid timestamps: 0
🚀 Future timestamps: 0

📏 Extremely long messages (>1000 chars): 0
📏 Extremely short messages (<3 chars): 0

✅ Summary:
  - Total Messages: 3349
  - Missing Messages: 0
  - Duplicate Ids: 0
  - Duplicate Texts: 0
  - User Inconsistencies: 0
  - Invalid Timestamps: 0
  - Future Timestamps: 0
  - Outlier Short Msgs: 0
  - Outlier Long Msgs: 0
